In [1]:
import pymongo
import time
import threading
import random

In [2]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
arr1 = db["arr1"]
arr2 = db["arr2"]
arr4 = db["arr4"]
arr8 = db["arr8"]
coll_list = [arr1, arr2, arr4, arr8]

In [4]:
user_list = [1, 2, 4]
depth_list = [1, 2, 4, 8]

In [98]:
select_1 = {"a1.device": {"$gte": 9480}}
select_2 = {"a1.a2.device": {"$gte": 9740}}
select_4 = {"a1.a2.a3.a4.device": {"$gte": 9930}}
select_8 = {"a1.a2.a3.a4.a5.a6.a7.a8.device": {"$gte": 9995}}
select_list = [select_1, select_2, select_4, select_8]

In [99]:
for d in range(len(depth_list)):
    print(coll_list[d].count_documents(select_list[d]))

1047
1022
1033
1167


In [6]:
def measure_read(user_id, duration_list, coll, whereClause):
    duration = 0
    num_of_tries = 1
    for i in range(num_of_tries):
        start = time.perf_counter()
        result = list(coll.find(whereClause))
        end = time.perf_counter()
        duration += end-start
    duration = (duration / num_of_tries) * 1000
    duration_list[user_id] = duration        

In [7]:
def test_read(num_of_users, depth):
    threads_list = []
    duration_list = [None] * num_of_users
    whereClause = select_list[depth]
    coll = coll_list[depth]

    for x in range(num_of_users):
        t = threading.Thread(target=measure_read, args=(x, duration_list, coll, whereClause,))
        t.start()
        threads_list.append(t)

    for t in threads_list:
        t.join()

    result = sum(duration_list)/ len(duration_list)
    return result

In [23]:
def execute_read():
    u_len = len(user_list)
    d_len = len(depth_list)
    result = [[0 for _ in range(d_len)] for _ in range(u_len)]

    for u in range(u_len):
        print(f"user: {user_list[u]}")
        print("depth: ", end=" ")
        for d in range(d_len):
            result[u][d] = test_read(user_list[u], d)
            print(f"{depth_list[d]}", end=" ")
        print("---")
    return result

In [119]:
def calculate_diffs(old, new):
    for u in range(len(user_list)):
        print(f"user {user_list[u]}")
        for d in range(len(depth_list)):
            o_val = old[u][d]
            n_val = new[u][d]
            diff = ((n_val - o_val) / o_val) * 100
            print(f"| {depth_list[d]} |{o_val:10.0f}| {n_val:10.0f}| {diff:10.1f} |")
        print("--------")

In [11]:
def createIndexes():
    coll_list[0].create_index([("a1.device", 1)])
    coll_list[1].create_index([("a1.a2.device", 1)])
    coll_list[2].create_index([("a1.a2.a3.a4.device", 1)])
    coll_list[3].create_index([("a1.a2.a3.a4.a5.a6.a7.a8.device", 1)])

In [12]:
def clearIndexes():
    coll_list[0].drop_indexes()
    coll_list[1].drop_indexes()
    coll_list[2].drop_indexes()
    coll_list[3].drop_indexes()

In [13]:
def showIndexes():
    for i in coll_list[0].list_indexes():
        print(i)
    for i in coll_list[1].list_indexes():
        print(i)
    for i in coll_list[2].list_indexes():
        print(i)
    for i in coll_list[3].list_indexes():
        print(i)

In [102]:
showIndexes()

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


In [101]:
clearIndexes()

In [103]:
no_index = execute_read()

user: 1
depth:  1 2 4 8 ---
user: 2
depth:  1 2 4 8 ---
user: 4
depth:  1 2 4 8 ---


In [104]:
createIndexes()

In [105]:
with_index = execute_read()

user: 1
depth:  1 2 4 8 ---
user: 2
depth:  1 2 4 8 ---
user: 4
depth:  1 2 4 8 ---


In [108]:
calculate_diffs(no_index, with_index)

user 1
depth: 1,      22.01      11.27      -48.8%
depth: 2,      28.15      11.67      -58.5%
depth: 4,     140.08      41.24      -70.6%
depth: 8,    2686.92    1096.13      -59.2%
--------
user 2
depth: 1,      18.59      10.68      -42.5%
depth: 2,      45.80      17.27      -62.3%
depth: 4,     243.04      99.52      -59.1%
depth: 8,    4411.13    2453.23      -44.4%
--------
user 4
depth: 1,      30.67      14.14      -53.9%
depth: 2,     118.70      20.47      -82.8%
depth: 4,     356.27      89.63      -74.8%
depth: 8,    7529.25    4001.31      -46.9%
--------


In [114]:
calculate_diffs(no_index, with_index)

user 1
depth: 1,         22         11      -48.8%
depth: 2,         28         12      -58.5%
depth: 4,        140         41      -70.6%
depth: 8,       2687       1096      -59.2%
--------
user 2
depth: 1,         19         11      -42.5%
depth: 2,         46         17      -62.3%
depth: 4,        243        100      -59.1%
depth: 8,       4411       2453      -44.4%
--------
user 4
depth: 1,         31         14      -53.9%
depth: 2,        119         20      -82.8%
depth: 4,        356         90      -74.8%
depth: 8,       7529       4001      -46.9%
--------


In [120]:
calculate_diffs(no_index, with_index)

user 1
| 1 |        22|         11|      -48.8 |
| 2 |        28|         12|      -58.5 |
| 4 |       140|         41|      -70.6 |
| 8 |      2687|       1096|      -59.2 |
--------
user 2
| 1 |        19|         11|      -42.5 |
| 2 |        46|         17|      -62.3 |
| 4 |       243|        100|      -59.1 |
| 8 |      4411|       2453|      -44.4 |
--------
user 4
| 1 |        31|         14|      -53.9 |
| 2 |       119|         20|      -82.8 |
| 4 |       356|         90|      -74.8 |
| 8 |      7529|       4001|      -46.9 |
--------


In [84]:
createIndexes()